<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [109]:
!pip3 install folium
!pip3 install wget
!pip3 install pandas

In [110]:
import folium
import wget
import pandas as pd

In [111]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [112]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.


In [113]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [114]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [115]:
import pandas as pd
import folium
from folium.features import DivIcon

# Load the SpaceX dataset
# Uncomment the next line to load the dataset if not already available
# spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Select relevant sub-columns
# spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Create a DataFrame of unique launch sites
# launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
# launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Create a map centered at a general coordinate (e.g., first launch site)
initial_coord = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]
site_map = folium.Map(location=initial_coord, zoom_start=5)

# Add a circle and a marker for each launch site
for _, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    lat, long = row['Lat'], row['Long']
    
    # Add a yellow circle to represent the launch site
    circle = folium.Circle(
        [lat, long],
        radius=1000,
        color='#FFFF00',  # Yellow border color
        fill=True,
        fill_color='#FFFF00',  # Yellow fill color
        fill_opacity=0.5
    ).add_child(folium.Popup(launch_site))
    site_map.add_child(circle)
    
    # Add a marker with the launch site's name in yellow
    marker = folium.Marker(
        [lat, long],
        icon=DivIcon(
            icon_size=(200,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#FFFF00;"><b>{launch_site}</b></div>',  # Yellow text
        )
    )
    site_map.add_child(marker)

# Display the map
site_map


and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [116]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [117]:
marker_cluster = MarkerCluster()

_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [118]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Load the SpaceX dataset
#spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")

# Create a map centered at an average coordinate
initial_coord = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=initial_coord, zoom_start=5)

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add a marker for each launch record
for _, row in spacex_df.iterrows():
    launch_site = row['Launch Site']
    lat, long = row['Lat'], row['Long']
    outcome = row['class']  # Launch success (1) or failure (0)

    # Set marker color based on class
    if outcome == 1:
        marker_color = 'green'
    else:
        marker_color = 'red'

    # Create a marker for the launch record
    folium.Marker(
        location=[lat, long],
        icon=folium.Icon(color=marker_color, icon='ok-sign'),  # Green or red marker
        popup=f"Launch Site: {launch_site}<br>Outcome: {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Display the map
site_map


In [119]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [133]:
#TODO: For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster
# Add marker_cluster to current site_map
#site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
#for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
   # marker_cluster.add_child(marker)

#site_map

import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Create a map centered around the average coordinates
map_center = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=map_center, zoom_start=4)  # Adjust zoom level for a wider view

# Initialize a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add markers for SpaceX launch results
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],  # Use latitude and longitude
        popup=row['Launch Site'],           # Display the launch site name in the popup
        icon=folium.Icon(color=row['marker_color'])  # Use marker_color column for icon color
    ).add_to(marker_cluster)

# Add circles and study site markers
for _, row in launch_sites_df.iterrows():
    launch_site = row['Launch Site']
    lat, long = row['Lat'], row['Long']
    
    # Add a yellow circle to represent the launch site
    circle = folium.Circle(
        [lat, long],
        radius=1000,
        color='#FFFF00',  # Yellow border color
        fill=True,
        fill_color='#FFFF00',  # Yellow fill color
        fill_opacity=0.5
    ).add_child(folium.Popup(launch_site))
    site_map.add_child(circle)
    
    # Add a study site name marker below the launch site marker
    marker = folium.Marker(
        [lat, long],
        icon=DivIcon(
            icon_size=(200, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#FFFF00;"><b>{launch_site}</b></div>'  # Yellow text for study site
        )
    )
    site_map.add_child(marker)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [134]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [135]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [136]:
!pip install geopy


_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [140]:
#TODO: Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)


import folium
from geopy.distance import geodesic

# Example coordinates for a launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356
launch_site_coords = (launch_site_lat, launch_site_lon)

# Initialize the map with MousePosition control
site_map = folium.Map(location=launch_site_coords, zoom_start=10)

# Add a launch site marker
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Add MousePosition to the map for interactive coordinate retrieval
folium.plugins.MousePosition().add_to(site_map)

# Coordinate of the closest coastline (e.g., obtained from MousePosition or manual selection)
coastline_lat = 28.56367
coastline_lon = -80.57163
coastline_coords = (coastline_lat, coastline_lon)

# Mark the closest coastline point
folium.Marker(
    location=coastline_coords,
    popup="Closest Coastline Point",
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# Draw a line between the launch site and the coastline
folium.PolyLine([launch_site_coords, coastline_coords], color="red").add_to(site_map)

# Calculate the distance between the launch site and the coastline
distance_coastline = geodesic(launch_site_coords, coastline_coords).kilometers

# Display the distance
print(f"Distance between the launch site and coastline: {distance_coastline:.2f} km")

# Save and display the map
#site_map.save("launch_site_coastline_map.html")
#print("Map saved as launch_site_coastline_map.html. Open it in a browser to view.")


Distance between the launch site and coastline: 0.58 km


_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [141]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

import folium
from folium.features import DivIcon
from geopy.distance import geodesic

# Example coordinates for a launch site and the closest coastline
launch_site_coords = (28.562302, -80.577356)  # Replace with actual launch site coordinates
coastline_coords = (28.56367, -80.57163)  # Replace with actual closest coastline coordinates

# Initialize the map centered at the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=15)

# Add a marker for the launch site
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Add a marker for the closest coastline point
folium.Marker(
    location=coastline_coords,
    popup="Closest Coastline Point",
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# Draw a line between the launch site and the coastline point
folium.PolyLine([launch_site_coords, coastline_coords], color="red").add_to(site_map)

# Calculate the distance between the launch site and the coastline
distance_km = geodesic(launch_site_coords, coastline_coords).kilometers

# Add a marker to display the distance on the map
distance_marker = folium.Marker(
    location=coastline_coords,
    icon=DivIcon(
        icon_size=(250, 36),  # Adjusted size for better visibility
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance_km:.2f} KM</b></div>'
    )
)
site_map.add_child(distance_marker)

# Save the map to an HTML file
#site_map.save("launch_site_with_distance_map.html")

# Print the distance
#print(f"Distance between the launch site and the closest coastline: {distance_km:.2f} KM")


_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [149]:
#TODO: Draw a PolyLine between a launch site to the selected coastline point
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
#lines=folium.PolyLine(locations=coordinates, weight=1)
#site_map.add_child(lines)
import folium

# Example coordinates for the launch site and the closest coastline point
launch_site_coords = (28.562302, -80.577356)  # Replace with your actual launch site coordinates
coastline_coords = (28.56367, -80.57163)  # Replace with actual coastline coordinates

# Initialize the map centered near the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=15)

# Add a marker for the launch site
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Add a marker for the closest coastline point
folium.Marker(
    location=coastline_coords,
    popup="Closest Coastline Point",
    icon=folium.Icon(color='green', icon='info-sign')
).add_to(site_map)

# Create a PolyLine to connect the launch site and coastline point
coordinates = [launch_site_coords, coastline_coords]  # Points to connect
line_to_coast = folium.PolyLine(
    locations=coordinates,
    weight=3,  # Thickness of the line
    color='red',  # Color of the line
    opacity=0.8  # Transparency level of the line
)
site_map.add_child(line_to_coast)

# Display the map
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [148]:
import folium
from folium.features import DivIcon
from geopy.distance import geodesic

# Example coordinates for a launch site (Replace with actual coordinates)
launch_site_coords = (28.562302, -80.577356)  # Replace with your launch site coordinates

# Coordinates for the closest city, railway, highway, etc.
closest_city_coords = (28.53834, -81.379236)  # Replace with coordinates obtained for the closest city
closest_railway_coords = (28.562167, -80.570833)  # Replace with coordinates obtained for the closest railway
closest_highway_coords = (28.563610, -80.569444)  # Replace with coordinates obtained for the closest highway

# Initialize the map centered at the launch site
site_map = folium.Map(location=launch_site_coords, zoom_start=12)

# Add marker for the launch site
folium.Marker(
    location=launch_site_coords,
    popup="Launch Site",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Create a function to add markers, calculate distances, and draw lines
def add_marker_and_line(site_coords, location_coords, name, map_obj):
    # Calculate distance
    distance_km = geodesic(site_coords, location_coords).kilometers

    # Add marker for the location
    folium.Marker(
        location=location_coords,
        icon=DivIcon(
            icon_size=(250, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{name}: {distance_km:.2f} KM</b></div>'
        )
    ).add_to(map_obj)

    # Draw a line between the launch site and the location
    folium.PolyLine(
        locations=[site_coords, location_coords],
        weight=2,
        color='red'
    ).add_to(map_obj)

# Add markers and lines for the closest city, railway, and highway
add_marker_and_line(launch_site_coords, closest_city_coords, "Closest City", site_map)
add_marker_and_line(launch_site_coords, closest_railway_coords, "Closest Railway", site_map)
add_marker_and_line(launch_site_coords, closest_highway_coords, "Closest Highway", site_map)

# Display the map directly within the Python environment
site_map


## After  plotting distance lines to the proximities, it is obviuos that not all the launch sites are in close proximity to either railways, highways or coastline. They also keep a certain distance from cities. This is due to security since it is not sure whether the launch will be successful or not.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
